# Preprocessor Notebook : Logements Sociaux, fichier RPLS annuel

Ce notebook traite le fichier Excel du RPLS annuel : données sur les logements sociaux.
Le but est de récupérer les datasets suivants, à partir du fichier XSLX téléchargé depuis le site du ministère du Développement Durable :
 - Données par régions
 - Données par départements
 - Données par EPCI
 - Données par communes

 ### Paramètres
 Ce Notebook prend des paramètres en entrée, définis sur la toute première cellule (ci-dessus).
 La cellule a le tag "parameters" ce qui permet de lui passer des valeurs via papermill.
 - filepath : le chemin vers le fichier Excel à traiter
 - model_name : le nom du modèle source

 ### Principe
 Ce notebook extrait 4 feuilles du fichier Excel d'entrée : region, departement, epci, communes. 
 Chaque feuille est chargée dans un dataFrame puis sauvegardée en .xlsx, et chargée en base de données Bronze.
 Peu de retraitement sur ces dataFrames, seul le tableau "departement" a besoin de renommer une colonne.

## Initialisation

Les cellules suivantes servent à importer les modules nécessaires et à préparer les variables communes utilisées dans les traitements.

In [1]:
# Baseline imports
import pandas as pd
import os
import sys
import datetime
from dotenv import dotenv_values
import sqlalchemy

# Dirty trick to be able to import common odis modules, if the notebook is not executed from 13_odis
current_dir = os.getcwd()
parent_dir = os.path.dirname(os.getcwd())
while not current_dir.endswith("13_odis"):
    print("changing to parent dir")
    os.chdir(parent_dir)
    current_dir = parent_dir
    parent_dir = os.path.dirname(current_dir)

print(os.getcwd())
sys.path.append(current_dir)

/Users/alex/dev/13_odis


In [2]:
# additional imports
from common.config import load_config
from common.data_source_model import DataSourceModel
from common.utils.file_handler import FileHandler
from common.utils.interfaces.data_handler import OperationType

## Paramètres du Notebook
Paramètres pouvant être passés en input par papermill.

Seuls des types built-in semblent marcher (str, int etc), les classes spécifiques ou les objets mutables (datetime...) semblent faire planter papermill.

Doc officielle de papermill : parametrize [https://papermill.readthedocs.io/en/latest/usage-parameterize.html]

In [3]:
# Define parameters for papermill. 
filepath = 'data/imports/logement/logement.logements_sociaux_1.xlsx'
model_name = "logement.logements_sociaux"


In [4]:
# Parameters
filepath = "data/imports/logement/logement.logements_sociaux_1.xlsx"
model_name = "logement.logements_sociaux"


In [5]:
# Initialize common variables
dataframes = {}
artifacts = []

start_time = datetime.datetime.now(tz=datetime.timezone.utc)
config = load_config("datasources.yaml", response_model=DataSourceModel)
model = config.get_model( model_name = model_name )
# Instantiate File Handler for file loads and dumps
handler = FileHandler()

## Traitement des données
A partir de là, on charge le fichier Excel dans Pandas et on traite les feuilles à récupérer, une par une

In [6]:
# Load workbook to pandas
wb = pd.ExcelFile(
    filepath,
    engine = 'openpyxl'
)

In [7]:
# Load excel sheet for Regions
sheet_name = "REGION"
keep_columns_region = [
    'LIBREG',
    'densite',
    'nb_ls',
    'tx_vac',
    'tx_mob'
]


df_region = pd.read_excel(wb, 
                    sheet_name = "REGION",
                    index_col = "REG",
                    header = 5
                    )

df_region = df_region[keep_columns_region]
dataframes["REGION"] = df_region

region_artifact = handler.artifact_dump( df_region, "REGION", model)
artifacts.append(region_artifact)

df_region.head()

2025-04-08 22:53:19,628 - main - INFO :: file_handler.py :: logement.logements_sociaux -> results saved to : 'data/imports/logement/logement.logements_sociaux_REGION.xlsx'


,LIBREG,densite,nb_ls,tx_vac,tx_mob
REG,,,,,
1,Guadeloupe,22.67,40059,4.0576,6.4367
2,Martinique,20.99,35441,3.3365,5.7104
3,Guyane,25.52,21762,5.8323,7.6475
4,La Réunion,23.97,83463,1.3322,6.0356
6,Mayotte,4.66,2941,9.9919,16.8759


In [8]:
# Load excel sheet for Departments
keep_columns_departments = [
    'Unnamed: 1',
    'densite',
    'nb_ls',
    'tx_vac',
    'tx_mob'
]

df_department = pd.read_excel(wb, 
                    sheet_name = "DEPARTEMENT",
                    index_col = "DEP",
                    header = 5
                    )

df_department = df_department[keep_columns_departments]

# TODO : rename column for Unnamed: 1

dataframes["DEPARTEMENT"] = df_department

department_artifact = handler.artifact_dump( df_department, "DEPARTEMENT", model)
artifacts.append(department_artifact)

df_department.head()

2025-04-08 22:53:19,768 - main - INFO :: file_handler.py :: logement.logements_sociaux -> results saved to : 'data/imports/logement/logement.logements_sociaux_DEPARTEMENT.xlsx'


,Unnamed: 1,densite,nb_ls,tx_vac,tx_mob
DEP,,,,,
01,Ain,17.41,49608,2.5346,9.4306
02,Aisne,17.72,41217,3.5452,10.1188
03,Allier,12.14,19853,5.1155,10.7542
04,Alpes-de-Haute-Provence,9.82,7888,2.0628,10.8499
05,Hautes-Alpes,11.80,8050,3.7698,7.6982


In [9]:
# Load excel sheet for EPCI
keep_columns_epci = [
    'LIBEPCI',
    'densite',
    'nb_ls',
    'tx_vac',
    'tx_mob'
]

df_epci = pd.read_excel(wb, 
                    sheet_name = "EPCI",
                    index_col = "EPCI_DEP",
                    header = 5
                    )

df_epci = df_epci[keep_columns_epci]

dataframes["EPCI"] = df_epci

epci_artifact = handler.artifact_dump( df_epci, "EPCI", model)
artifacts.append(epci_artifact)

df_epci.head()

2025-04-08 22:53:21,463 - main - INFO :: file_handler.py :: logement.logements_sociaux -> results saved to : 'data/imports/logement/logement.logements_sociaux_EPCI.xlsx'


,LIBEPCI,densite,nb_ls,tx_vac,tx_mob
EPCI_DEP,,,,,
200029999 - (01),CC Rives de l'Ain - Pays du Cerdon,9.90,637,2.7553,11.8506
200040350 - (01),CC Bugey Sud,12.68,1936,3.5656,9.9287
200040590 - (01),CA Villefranche Beaujolais Saône,26.53,713,1.8545,8.4165
200042497 - (01),CC Dombes Saône Vallée,12.37,1976,1.5041,10.2096
200042935 - (01),CA Haut - Bugey Agglomération,29.97,8178,3.7475,9.5453


In [10]:
# Load excel sheet for COMMUNES
keep_columns_communes = [
    'LIBCOM_DEP',
    'densite',
    'nb_ls',
    'tx_vac',
    'tx_mob'
]

df_communes = pd.read_excel(wb, 
                    sheet_name = "COMMUNES",
                    index_col = "DEPCOM_ARM",
                    header = 5
                    )

df_communes = df_communes[keep_columns_communes]

dataframes["COMMUNES"] = df_communes

communes_artifact = handler.artifact_dump( df_communes, "COMMUNES", model )
artifacts.append(communes_artifact)

df_communes.head()

2025-04-08 22:53:58,015 - main - INFO :: file_handler.py :: logement.logements_sociaux -> results saved to : 'data/imports/logement/logement.logements_sociaux_COMMUNES.xlsx'


,LIBCOM_DEP,densite,nb_ls,tx_vac,tx_mob
DEPCOM_ARM,,,,,
01001,L'Abergement-Clémenciat (01),9.38,32,3.1250,9.0909
01004,Ambérieu-en-Bugey (01),30.42,2109,4.7952,10.1169
01005,Ambérieux-en-Dombes (01),14.22,113,2.7273,14.8515
01007,Ambronay (01),10.75,129,2.4390,12.1951
01008,Ambutrix (01),5.12,17,5.8824,5.8824


## Sauvegarde des métadonnées
On sauvegarde les métadonnées du processus localement, pour garder l'historique et pouvoir reprendre après erreur si besoin

In [11]:
for artifact in artifacts:
    print(artifact.model_dump( mode = "json" ))

preprocess_metadata = handler.dump_metadata(
    model = model,
    operation = OperationType.PREPROCESS,
    start_time = start_time,
    complete = True,
    errors = 0,
    artifacts = artifacts,
    pages = []
)

{'name': 'REGION', 'storage_info': {'location': 'data/imports/logement', 'format': 'xlsx', 'file_name': 'logement.logements_sociaux_REGION.xlsx', 'encoding': 'utf-8'}, 'load_to_bronze': True, 'success': True}
{'name': 'DEPARTEMENT', 'storage_info': {'location': 'data/imports/logement', 'format': 'xlsx', 'file_name': 'logement.logements_sociaux_DEPARTEMENT.xlsx', 'encoding': 'utf-8'}, 'load_to_bronze': True, 'success': True}
{'name': 'EPCI', 'storage_info': {'location': 'data/imports/logement', 'format': 'xlsx', 'file_name': 'logement.logements_sociaux_EPCI.xlsx', 'encoding': 'utf-8'}, 'load_to_bronze': True, 'success': True}
{'name': 'COMMUNES', 'storage_info': {'location': 'data/imports/logement', 'format': 'xlsx', 'file_name': 'logement.logements_sociaux_COMMUNES.xlsx', 'encoding': 'utf-8'}, 'load_to_bronze': True, 'success': True}
2025-04-08 22:53:58,052 - main - INFO :: file_handler.py :: logement.logements_sociaux -> results saved to : 'data/imports/logement/logement.logements_soc

2025-04-08 22:53:58,052 - main - DEBUG :: file_handler.py :: Metadata written in: 'data/imports/logement/logement.logements_sociaux_metadata_preprocess.json'


## Chargement en couche Bronze
On charge un engine SQLAchemy pour charger tous les datasets en base

In [12]:
# prepare db client
vals = dotenv_values()

conn_str = "postgresql://{}:{}@{}:{}/{}".format(
    vals["PG_DB_USER"],
    vals["PG_DB_PWD"],
    vals["PG_DB_HOST"],
    vals["PG_DB_PORT"],
    vals["PG_DB_NAME"]
)

dbengine = sqlalchemy.create_engine(conn_str)

In [13]:
# insert all to bronze
# make the final table name lowercase to avoid issues in Postgre
for name, dataframe in dataframes.items():
    dataframe.to_sql(
        name = f"{model.table_name}_{name.lower()}",
        con = dbengine,
        schema = 'bronze',
        index = True,
        if_exists = 'replace'
    )
